# Running Kriging Compute

This notebook demonstrates a complete geostatistical workflow:

1. Load drill hole assay data as a **PointSet**
2. Define a **Variogram** model for spatial correlation
3. **Visualize** the pointset and variogram together with Plotly
4. Run **kriging estimation** using Evo Compute (WIP)

## Requirements

You must have a Seequent account with the Evo entitlement. You'll need:
- The client ID of your Evo application
- The callback/redirect URL of your Evo application

To obtain these credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens).

## 1. Authentication

Authenticate using the `ServiceManagerWidget` which handles OAuth login and workspace selection.

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Replace with your Evo app credentials
client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id,
    redirect_url=redirect_url,
    cache_location="./notebook-data",
).login()

In [ ]:
# Load the widgets extension for rich HTML display
%load_ext evo.widgets

## 2. Load and Create PointSet from Drill Hole Data

The WP_assay.csv dataset contains:
- **8,332 sample points** from 55 drill holes
- **Coordinates**: X, Y, Z (EPSG:32650 - UTM Zone 50N)
- **Attributes**: Hole ID, CU_pct (copper %), AU_gpt (gold g/t), DENSITY

In [ ]:
import pandas as pd

# Load the sample assay data
input_file = "sample-data/WP_assay.csv"
df = pd.read_csv(input_file)

print(f"Loaded {len(df)} sample points from {df['Hole ID'].nunique()} drill holes")
print(f"\nSpatial extent:")
print(f"  X: {df['X'].min():.1f} to {df['X'].max():.1f} ({df['X'].max() - df['X'].min():.1f}m)")
print(f"  Y: {df['Y'].min():.1f} to {df['Y'].max():.1f} ({df['Y'].max() - df['Y'].min():.1f}m)")
print(f"  Z: {df['Z'].min():.1f} to {df['Z'].max():.1f} ({df['Z'].max() - df['Z'].min():.1f}m)")
print(f"\nCopper (CU_pct) statistics:")
print(f"  Mean: {df['CU_pct'].mean():.3f}%, Variance: {df['CU_pct'].var():.3f}")
df.head()

In [ ]:
from evo.objects.typed import PointSet, PointSetData, EpsgCode

# Prepare the DataFrame with required column names (lowercase x, y, z)
locations_df = df.rename(columns={"X": "x", "Y": "y", "Z": "z"})

# Create the pointset data
pointset_data = PointSetData(
    name="WP Drill Hole Assays",
    description="Copper and gold assay data from 55 drill holes",
    locations=locations_df,
    coordinate_reference_system=EpsgCode(32650),  # UTM Zone 50N
)

# Create the pointset in Evo
pointset = await PointSet.create(manager, pointset_data)
print(f"Created pointset with {pointset.num_points} points")

In [ ]:
# Display the pointset with rich HTML formatting
pointset

In [ ]:
# View available attributes
pointset.attributes

## 3. Create a Variogram Model

Variograms are geostatistical models that describe spatial correlation structure. They are fundamental to kriging interpolation and resource estimation.

A variogram consists of:
- **Nugget**: The variance at zero lag (y-intercept), representing measurement error or micro-scale variation
- **Sill**: The total variance (plateau value)
- **Structures**: Mathematical models (spherical, exponential, etc.) that define how correlation decreases with distance
- **Anisotropy**: Directional variation in correlation ranges

We define a nested spherical variogram for the copper grades (CU_pct) with:

- **Sill**: 0.84 (total variance of the copper data)
- **Nugget**: 0.08 (~10% nugget effect for measurement error)
- **Two nested structures**:
  - Short-range: 30% contribution, ranges 80m × 60m × 40m
  - Long-range: 60% contribution, ranges 250m × 180m × 100m

The anisotropy is aligned with the dominant NNE-SSW trend of the drill holes:
- Dip azimuth: 15° (strike direction)
- Dip: 70° (steep dip to the east)
- Pitch: 0°

In [ ]:
from evo.objects.typed import (
    Ellipsoid,
    EllipsoidRanges,
    Rotation,
    SphericalStructure,
    Variogram,
    VariogramData,
)

# Define the variogram model for copper grades
variogram_data = VariogramData(
    name="CU_pct Variogram",
    description="Nested spherical variogram for copper grades from WP drill holes",
    sill=0.84,  # Total variance (nugget + all contributions must equal sill)
    nugget=0.08,  # ~10% nugget effect
    is_rotation_fixed=True,  # All structures share the same rotation
    modelling_space="data",  # Original units (not normal score transformed)
    data_variance=0.84,  # Variance of the input data
    attribute="CU_pct",  # The attribute this variogram models
    structures=[
        # Short-range structure (30% of sill variance)
        SphericalStructure(
            contribution=0.25,  # nugget(0.08) + 0.25 + 0.51 = 0.84
            anisotropy=Ellipsoid(
                ranges=EllipsoidRanges(
                    major=80.0,   # 80m in major (along strike)
                    semi_major=60.0,  # 60m in semi-major (across strike)
                    minor=40.0,   # 40m in minor (vertical)
                ),
                rotation=Rotation(
                    dip=70.0,       # Steep dip
                    dip_azimuth=15.0,  # NNE strike direction
                    pitch=0.0,
                ),
            ),
        ),
        # Long-range structure (60% of sill variance)
        SphericalStructure(
            contribution=0.51,
            anisotropy=Ellipsoid(
                ranges=EllipsoidRanges(
                    major=250.0,  # 250m in major direction
                    semi_major=180.0,  # 180m in semi-major direction
                    minor=100.0,  # 100m in minor direction
                ),
                rotation=Rotation(
                    dip=70.0,
                    dip_azimuth=15.0,
                    pitch=0.0,
                ),
            ),
        ),
    ],
)

# Create the variogram object in Evo
variogram = await Variogram.create(manager, variogram_data)
print(f"Created variogram: {variogram.name}")

In [ ]:
# Display the variogram with rich HTML formatting
variogram

### Inspecting Variogram Properties

In [ ]:
print(f"Variogram: {variogram.name}")
print(f"Sill: {variogram.sill}")
print(f"Nugget: {variogram.nugget}")
print(f"Number of structures: {len(variogram.structures)}")
print(f"Modelling space: {variogram.modelling_space}")
print(f"Attribute: {variogram.attribute}")

# Inspect each structure
for i, struct in enumerate(variogram.structures):
    vtype = struct.get("variogram_type")
    contribution = struct.get("contribution")
    ranges = struct.get("anisotropy", {}).get("ellipsoid_ranges", {})
    print(f"\nStructure {i + 1}: {vtype}")
    print(f"  Contribution: {contribution}")
    print(f"  Ranges: major={ranges.get('major')}m, semi_major={ranges.get('semi_major')}m, minor={ranges.get('minor')}m")

## 4. Visualize Variogram Curves

The `get_principal_directions()` method returns variogram curves for plotting along the three principal axes.

In [ ]:
# Get variogram curves for the three principal directions
major, semi_major, minor = variogram.get_principal_directions()

print(f"Major direction: range={major.range_value}m, sill={major.sill}")
print(f"Semi-major direction: range={semi_major.range_value}m")
print(f"Minor direction: range={minor.range_value}m")
print(f"Points per curve: {len(major.distance)}")

In [ ]:
import plotly.graph_objects as go

# Create variogram curve plot
fig = go.Figure()

# Add curves for each principal direction
fig.add_trace(
    go.Scatter(
        x=minor.distance,
        y=minor.semivariance,
        name=f"Minor (range={minor.range_value:.0f}m)",
        line=dict(color="blue", width=2),
    )
)

fig.add_trace(
    go.Scatter(
        x=semi_major.distance,
        y=semi_major.semivariance,
        name=f"Semi-major (range={semi_major.range_value:.0f}m)",
        line=dict(color="green", width=2),
    )
)

fig.add_trace(
    go.Scatter(
        x=major.distance,
        y=major.semivariance,
        name=f"Major (range={major.range_value:.0f}m)",
        line=dict(color="red", width=2),
    )
)

# Add reference lines for nugget and sill
fig.add_hline(
    y=variogram.nugget,
    line_dash="dash",
    line_color="gray",
    annotation_text="Nugget",
    annotation_position="right",
)
fig.add_hline(
    y=variogram.sill,
    line_dash="dash",
    line_color="black",
    annotation_text="Sill",
    annotation_position="right",
)

fig.update_layout(
    title="CU_pct Variogram Model - Principal Directions",
    xaxis_title="Lag Distance (m)",
    yaxis_title="Semivariance γ(h)",
    legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99),
    template="plotly_white",
)

fig.show()

## 5. Working with Ellipsoids

Ellipsoids represent 3D spatial regions and are used for:
- **Search neighborhoods** in kriging - defining which samples influence each estimated point
- **Variogram anisotropy** - visualizing the directional ranges of spatial correlation

In [ ]:
# Get the ellipsoid from the variogram (uses structure with largest volume by default)
var_ellipsoid = variogram.get_ellipsoid()

print("Variogram ellipsoid ranges:")
print(f"  Major: {var_ellipsoid.ranges.major}m")
print(f"  Semi-major: {var_ellipsoid.ranges.semi_major}m")
print(f"  Minor: {var_ellipsoid.ranges.minor}m")

# Create a search ellipsoid scaled by 2x (typical for kriging)
search_ellipsoid = var_ellipsoid.scaled(2.0)

print(f"\nSearch ellipsoid (2x): major={search_ellipsoid.ranges.major}m")

## 6. Visualize PointSet and Variogram Ellipsoid Together

Create a 3D visualization showing:
- Drill hole sample points colored by copper grade
- Variogram anisotropy ellipsoid at the data centroid
- Search ellipsoid (2× variogram range) for kriging

In [ ]:
# Get pointset data for visualization
points_df = await pointset.to_dataframe()

# Calculate centroid for ellipsoid placement
center = (
    points_df["x"].mean(),
    points_df["y"].mean(),
    points_df["z"].mean(),
)
print(f"Data centroid: ({center[0]:.1f}, {center[1]:.1f}, {center[2]:.1f})")

In [ ]:
import numpy as np

# Generate surface mesh points for visualization
vx, vy, vz = var_ellipsoid.surface_points(center=center, n_points=25)
sx, sy, sz = search_ellipsoid.surface_points(center=center, n_points=25)

# Create 3D figure
fig = go.Figure()

# Add sample points colored by CU_pct
fig.add_trace(
    go.Scatter3d(
        x=points_df["x"],
        y=points_df["y"],
        z=points_df["z"],
        mode="markers",
        marker=dict(
            size=2,
            color=points_df["CU_pct"],
            colorscale="Viridis",
            colorbar=dict(title="CU_pct (%)", x=1.02),
            cmin=0,
            cmax=3,  # Cap at 3% for better color distribution
        ),
        name="Drill Hole Samples",
        hovertemplate="X: %{x:.1f}<br>Y: %{y:.1f}<br>Z: %{z:.1f}<br>CU: %{marker.color:.2f}%<extra></extra>",
    )
)

# Add variogram ellipsoid as semi-transparent mesh
fig.add_trace(
    go.Mesh3d(
        x=vx,
        y=vy,
        z=vz,
        alphahull=0,
        opacity=0.3,
        color="blue",
        name=f"Variogram Ellipsoid ({var_ellipsoid.ranges.major:.0f}m)",
    )
)

# Add search ellipsoid as semi-transparent mesh
fig.add_trace(
    go.Mesh3d(
        x=sx,
        y=sy,
        z=sz,
        alphahull=0,
        opacity=0.15,
        color="gold",
        name=f"Search Ellipsoid (2×, {search_ellipsoid.ranges.major:.0f}m)",
    )
)

# Add centroid marker
fig.add_trace(
    go.Scatter3d(
        x=[center[0]],
        y=[center[1]],
        z=[center[2]],
        mode="markers",
        marker=dict(size=8, color="red", symbol="diamond"),
        name="Centroid",
    )
)

fig.update_layout(
    title="WP Drill Hole Data with Variogram Anisotropy",
    scene=dict(
        xaxis_title="Easting (m)",
        yaxis_title="Northing (m)",
        zaxis_title="Elevation (m)",
        aspectmode="data",
    ),
    template="plotly_white",
    showlegend=True,
)

fig.show()

## 7. View Objects in Evo

Generate URLs to view the created objects in the Evo Portal and Viewer.

In [ ]:
from evo.widgets import get_viewer_url_for_objects

# Generate a viewer URL to see both objects together
viewer_url = get_viewer_url_for_objects(manager, [pointset, variogram])
print(f"View in Evo Viewer: {viewer_url}")

---

## WIP: Creating Kriging and Compute

The following sections demonstrate how to run kriging estimation using Evo Compute.

**Note:** This functionality is under development. The code below shows the expected API pattern.

### WIP: Create Target Block Model

Create a Block Model to hold the kriging results. The block model defines the estimation grid.

In [ ]:
# WIP: Create target block model for kriging estimation
#
# from evo.objects.typed import BlockModel, RegularBlockModelData, Point3, Size3i, Size3d
# from evo.blockmodels import Units
#
# # Define block model covering the drill hole extent
# bm_data = RegularBlockModelData(
#     name="CU Kriging Estimate",
#     description="Block model with kriged copper grades",
#     origin=Point3(x=444750, y=492850, z=2350),
#     n_blocks=Size3i(nx=50, ny=75, nz=45),  # 50x75x45 blocks
#     block_size=Size3d(dx=20.0, dy=20.0, dz=20.0),  # 20m blocks
#     crs="EPSG:32650",
#     size_unit_id=Units.METRES,
# )
#
# block_model = await BlockModel.create_regular(manager, bm_data)
# print(f"Created Block Model: {block_model.name}")
# print(f"Bounding Box: {block_model.bounding_box}")

### WIP: Define Kriging Parameters

Configure the kriging search neighborhood and estimation parameters.

In [ ]:
# WIP: Define kriging parameters
#
# from evo.compute.tasks import SearchNeighborhood
# from evo.compute.tasks.kriging import KrigingParameters
#
# # Use the search ellipsoid we created earlier (2x variogram range)
# params = KrigingParameters(
#     source=pointset.attributes["CU_pct"],  # Source attribute
#     target=block_model.attributes["CU_estimate"],  # Target attribute
#     variogram=variogram,
#     search=SearchNeighborhood(
#         ellipsoid=search_ellipsoid,
#         max_samples=16,  # Maximum samples per estimate
#         min_samples=4,   # Minimum samples required
#     ),
# )
#
# print(f"Kriging source: {params.source}")
# print(f"Search ellipsoid: major={search_ellipsoid.ranges.major}m")

### WIP: Run Kriging Task

Submit and run the kriging task using Evo Compute.

In [ ]:
# WIP: Run kriging task
#
# from evo.compute.tasks import run
#
# # Submit kriging task (progress feedback is shown by default)
# print("Submitting kriging task...")
# results = await run(manager, [params])
#
# print(f"Kriging complete!")
# print(f"Result: {results[0].message}")

### WIP: View Kriging Results

Refresh the block model and view the estimated grades.

In [ ]:
# WIP: View kriging results
#
# # Refresh block model to see new attributes
# block_model = await block_model.refresh()
#
# # Display the block model (pretty-printed with Portal/Viewer links)
# block_model

In [ ]:
# WIP: Query estimated values
#
# # Get the kriged values as a DataFrame
# results_df = await block_model.to_dataframe(columns=["CU_estimate"])
#
# print(f"Estimated {len(results_df)} blocks")
# print(f"\nStatistics:")
# print(results_df["CU_estimate"].describe())

### WIP: Running Multiple Kriging Scenarios

Run multiple kriging tasks concurrently to compare different parameters.

In [ ]:
# WIP: Multiple kriging scenarios with different max_samples
#
# max_samples_values = [5, 10, 15, 20]
#
# # Create parameter sets for each scenario
# parameter_sets = []
# for max_samples in max_samples_values:
#     params = KrigingParameters(
#         source=pointset.attributes["CU_pct"],
#         target=block_model.attributes[f"CU_samples_{max_samples}"],
#         variogram=variogram,
#         search=SearchNeighborhood(
#             ellipsoid=search_ellipsoid,
#             max_samples=max_samples,
#         ),
#     )
#     parameter_sets.append(params)
#
# # Run all scenarios in parallel
# print(f"Submitting {len(parameter_sets)} kriging tasks...")
# results = await run(manager, parameter_sets)
# print(f"All {len(results)} scenarios completed!")